In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('/Users/EJ/dev/vmo/vmo/utils')
sys.path.append('/Users/EJ/dev/vmo/vmo/utils/nuxmv')
sys.path.append('/Users/EJ/dev/vmo/vmo/distances')
sys.path.append('/Users/EJ/dev/vmo/vmo/analysis')
sys.path.append('/Users/EJ/dev/vmo/vmo/logics')
sys.path.append('/Users/EJ/dev/vmo/vmo')
sys.path.append('/Users/EJ/dev/vmo/VMO')




# /Users/EJ/dev/vmo/vmo/utils
# chromagram.py  harmonic_changes.py  music21_interface.py


import chromagram as vchroma
import harmonic_changes as vharm
import music21_interface as vmusic
import model_checking as vmodel
import tonnetz as ton

#import analysis as van
#import generate as vge


import numpy as np
import os.path as path

import matplotlib.pyplot as plt
%matplotlib inline
import sklearn.preprocessing as pre
# import librosa
import music21 as mus

import vmo
import vmo.analysis as van
import vmo.generate as vge

# Music analysis
##import vmo.utils.chromagram as vchroma
##import vmo.utils.music21_interface as vmusic
##import vmo.utils.harmonic_changes as vharm

# Logics
##import vmo.logics.model_checking as vmodel

/Users/EJ/anaconda/lib/python2.7/site-packages/fuzzywuzzy/fuzz.py:35: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [6]:
## Midi file access
midi_path = path.expanduser('/Users/EJ/dev/vmo/bin/files/midi/parker_omnibook/')
titles = ['blues_for_alice', 'marmaduke',
          'now_s_the_time', 'ornithology']
def make_filename(title):
    midi_file = title + '.mid'
    return midi_path + midi_file
    
"""Oracle generation."""

# Generate music21 stream from MIDI file

# Choose whether all pieces should be concatenated as a corpus
concatenate = False

if concatenate:
    # Initialize new, empty stream
    notes = mus.stream.Stream()
    for title in titles:
        # Aggregate all four titles
        stream_full_band = mus.converter.parse(make_filename(title))
        stream_piano = stream_full_band[2]
        flat_piano = stream_piano.flat.notesAndRests.quantize()
        
        duration = flat_piano.duration.quarterLength
        delta_quadratura = duration % 4
        flat_piano.append(mus.note.Rest(delta_quadratura))
        
        notes.append(flat_piano.flat.notesAndRests.quantize())
    notes = notes.flat.notesAndRests.quantize()
    chosen_title = 'Concatenation'
else:
    chosen_title = titles[3]  # Choose among the proposed titles
    stream_full_band = mus.converter.parse(make_filename(chosen_title))
    stream_piano = stream_full_band[2]
    notes = stream_piano.flat.notesAndRests.quantize()

In [7]:
# Generate VMO from music21 stream

oracle = vmusic.from_stream(notes, framesize=4.0, dfunc='tonnetz', threshold=2)

/Users/EJ/dev/vmo/vmo/VMO/oracle.py:756: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  suffix_candidate = self.trn[k][earliest_close_feat]


In [8]:
notes.flat.notesAndRests.show('text')
chosen_title
# notes.show('midi')

{0.0} <music21.note.Rest rest>
{0.0} <music21.note.Rest rest>
{0.0} <music21.note.Rest rest>
{8.0} <music21.chord.Chord G3 B3 D4>
{8.0} <music21.note.Note G>
{8.0} <music21.note.Rest rest>
{12.75} <music21.note.Note G>
{13.0} <music21.note.Note B>
{13.0} <music21.note.Note B>
{13.0} <music21.note.Rest rest>
{13.0} <music21.note.Note D>
{13.75} <music21.chord.Chord B3 B4>
{13.75} <music21.note.Note D>
{14.75} <music21.note.Note G>
{15.0} <music21.note.Note B>
{15.0} <music21.note.Note B>
{15.0} <music21.note.Note D>
{16.0} <music21.note.Note G>
{17.0} <music21.chord.Chord G3 G4 B-3 D4>
{18.75} <music21.chord.Chord D4 B-3 G3>
{20.0} <music21.note.Note C>
{21.75} <music21.chord.Chord E4 C4 B-3 G4>
{24.0} <music21.chord.Chord F3 F4 A3 C4>
{24.0} <music21.note.Note F>
{28.0} <music21.note.Note F>
{28.0} <music21.chord.Chord F4 A3 C4>
{29.6667} <music21.chord.Chord F3 F4 A3 C4>
{30.6667} <music21.note.Note C>
{32.0} <music21.note.Note F>
{33.75} <music21.chord.Chord F4 C4 G#3>
{36.0} <music2

'ornithology'

In [9]:
"""Harmonic changes detection and chord progression extraction."""

# Extract the first 16 quarter-notes
_beginning_notes = vmusic.extract_frame(notes, 0., 32.)
# Compute offsets of harmonic changes (in quarter-length)
harmonic_changes = vharm.from_stream_by_offsets(
    _beginning_notes, framesize=1/4.)
# Compute the chord progression in the extracted frame
prog = vmusic.get_chord_progression(_beginning_notes)

print("First two measures:")
_beginning_notes.show('text')
print("\nExtracted progression:\n" + str(prog))
print("\nHarmonic changes:\n" + str(harmonic_changes))

First two measures:
{0.0} <music21.note.Rest rest>
{0.0} <music21.note.Rest rest>
{0.0} <music21.note.Rest rest>
{8.0} <music21.chord.Chord G3 B3 D4>
{8.0} <music21.note.Note G>
{8.0} <music21.note.Rest rest>
{12.75} <music21.note.Note G>
{13.0} <music21.note.Note B>
{13.0} <music21.note.Note B>
{13.0} <music21.note.Rest rest>
{13.0} <music21.note.Note D>
{13.75} <music21.chord.Chord B3 B4>
{13.75} <music21.note.Note D>
{14.75} <music21.note.Note G>
{15.0} <music21.note.Note B>
{15.0} <music21.note.Note B>
{15.0} <music21.note.Note D>
{16.0} <music21.note.Note G>
{17.0} <music21.chord.Chord G3 G4 B-3 D4>
{18.75} <music21.chord.Chord D4 B-3 G3>
{20.0} <music21.note.Note C>
{21.75} <music21.chord.Chord E4 C4 B-3 G4>
{24.0} <music21.chord.Chord F3 F4 A3 C4>
{24.0} <music21.note.Note F>
{28.0} <music21.note.Note F>
{28.0} <music21.chord.Chord F4 A3 C4>
{29.6667} <music21.chord.Chord F3 F4 A3 C4>
{30.6667} <music21.note.Note C>

Extracted progression:
['silence', 'silence', 'silence', 'sile

In [10]:
# Extract one frame from the oracle using the original stream
vmusic.extract_frame_oracle(notes.flat, oracle, 6).flat.show('text')

{20.0} <music21.note.Note C>
{21.75} <music21.chord.Chord E4 C4 B-3 G4>


In [11]:
"""Integrated MIDI playback.

Might need some parameters-tweaking to work if midi player binding is not
done straight out-of-the-box with music21.

In that case, run:
>>> from music21 import *
>>> us = environment.UserSettings()
>>> us.create()
>>> us['midiPath'] = <PATH/TO/MIDI/PLAYER>
>>> # Example : us['midiPath'] = u'/usr/bin/totem-audio-preview'
"""

# _beginning_notes.show('midi')

"Integrated MIDI playback.\n\nMight need some parameters-tweaking to work if midi player binding is not\ndone straight out-of-the-box with music21.\n\nIn that case, run:\n>>> from music21 import *\n>>> us = environment.UserSettings()\n>>> us.create()\n>>> us['midiPath'] = <PATH/TO/MIDI/PLAYER>\n>>> # Example : us['midiPath'] = u'/usr/bin/totem-audio-preview'\n"

In [13]:
"""Chord-progression extraction."""
# Two sequences of degrees. We will try to generate a path following the first
# then the second (possibibly with a non-empty arbitrary path between both) in `oracle`

prog_1 = [[1, 4, 7, 3, 6, 7, 3, 6]]
prog_2 = [[1, 4, 2, 5, 1, 6, 2, 5]]

gen_path_1, chosen_tonic_1 = vmodel.make_progression_from_degrees(
    oracle, prog_1, original_stream=notes, enable_motions=True)
gen_path_2, chosen_tonic_2 = vmodel.make_progression_from_degrees(
    oracle, prog_2, original_stream=notes, enable_motions=True)

def path_printer(path, tonic):
    if path:
        print("A path has successfully been generated!")
        print("The chosen tonic is: " + tonic)
        print("\nThe path is the following: \n")
        for state in path:
            print(state)
    else:
        print("No path was found.")

print("Path 1\n")
path_printer(gen_path_1, chosen_tonic_1)

print("\n===\n\nPath 2\n")
path_printer(gen_path_2, chosen_tonic_2)


/Users/EJ/dev/vmo/vmo/utils/music21_interface.py:168: Music21DeprecationWarning: _getPitchClass was deprecated on May 2014 and will disappear at or after May 2016. use pitch.pitchClass instead
  p_tonic = mus.note.Note(tonic).pitchClass


OSError: [Errno 13] Permission denied

In [52]:
def midiSave(stream, filename):
    mf = mus.midi.translate.streamToMidiFile(stream)
    mf.open(filename + '.mid', 'wb')
    mf.write()

# Recompose new stream from extracted path

if gen_path_1:
    extracted_1 = vmusic.path_to_stream(notes, gen_path_1[0:-1], framesize=4.0).quantize()
    midiSave(extracted_1, chosen_title + '_genpath_1')
    print(vmusic.get_chord_progression(extracted_1, framesize=4.0))
    
    # Plot piano roll of the extracted stream
    extracted_1.plot('pianoroll')
    
if gen_path_2:
    extracted_2 = vmusic.path_to_stream(notes, gen_path_2[0:-1], framesize=4.0).quantize()
    midiSave(extracted_2, chosen_title + '_genpath_2')
    print(vmusic.get_chord_progression(extracted_2, framesize=4.0))
    
    # Plot piano roll of the extracted stream
    extracted_2.plot('pianoroll')

[autoreload of vmo.VMO.oracle failed: Traceback (most recent call last):
  File "/Users/EJ/anaconda/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "/Users/EJ/anaconda/lib/python2.7/site-packages/vmo/VMO/oracle.py", line 160
    def _init_new_state(self):
    ^
IndentationError: unexpected indent
]


NameError: name 'gen_path_1' is not defined

In [ ]:
extracted_1.show('midi')

In [ ]:
extracted_2.show('midi')